In [1]:
import pandas as pd
import numpy as np
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import glob


In [2]:
sc = SparkContext()
sq = pyspark.sql.SQLContext(sc)

In [3]:
data = pd.read_csv('data_train.csv')

In [4]:
data.head()

,Id,Prediction
0,r44_c1,4
1,r61_c1,3
2,r67_c1,4
3,r72_c1,3
4,r86_c1,5


In [5]:
def cross_vad(y,ratio) :

    n = y.shape[0]
    interval = int(n *ratio)
    seed = 3
    np.random.seed(seed)
    index = np.random.permutation(n)
    k_train = index[0:interval]
    k_test  = index[interval:n]
    return k_train , k_test

In [6]:
dict_ ={'User':[],'Movie':[]}
for id_ in ((data).Id.values) :
    split = id_.split('_')
    dict_['User'] += [split[0][1:]]
    dict_['Movie'] += [split[1][1:]]
formated_data = pd.DataFrame(dict_)
formated_data.head()

,Movie,User
0,1,44
1,1,61
2,1,67
3,1,72
4,1,86


In [7]:
formated_data.loc[:,'Rating'] = data.Prediction

In [8]:
index_train, index_test = cross_vad(data,0.7)
train = formated_data.loc[index_train]
test  = formated_data.loc[index_test]
test.head()

,Movie,User,Rating
723429,606,2319,4
955994,761,9510,2
923421,731,9585,3
533152,458,7284,4
693011,594,2349,5


In [9]:
test_on = test.drop('Rating',1)

In [10]:
pDF = sq.createDataFrame(train)
spDF = pDF.rdd

In [11]:
# Build the recommendation model using Alternating Least Squares
lambdas = np.logspace(-10,-0.01,10)
#for lambda_ in lambdas :
rank = 6 
lambda_ = 0.07585775750291851
print('rank = {c} and lambda = {l}'.format(c=(rank),l=lambda_))
#for i in range(5,21) :

print('iterations = {c}'.format(c=(17)))
#rank = i+1
numIterations = 17
model = ALS.train(spDF, rank, numIterations,lambda_)
pDF_test = sq.createDataFrame(test_on)
spDF_test = pDF_test.rdd
testdata = spDF_test.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: (r[0], r[1], r[2]))
s  = predictions.toDF()
lol = s.toPandas()
lol.sort_values(['_1','_2'],ascending=[1,1],inplace=True)
predic_end = lol.reset_index(drop=True)
test.Movie = test.Movie.values.astype(int)
test.User = test.User.values.astype(int)
new_test = test.sort_values(by=['Movie','User'],ascending=[True,True])
test_end = new_test.reset_index(drop=True)
test_end.head()
print(np.mean((test_end.Rating.values-predic_end._3.values)**2))

rank = 6 and lambda = 0.07585775750291851
iterations = 17
0.983372374568


In [12]:
data_test = pd.read_csv('sampleSubmission.csv')

In [13]:
dict_ ={'User':[],'Movie':[]}
for id_ in ((data_test).Id.values) :
    split = id_.split('_')
    dict_['User'] += [split[0][1:]]
    dict_['Movie'] += [split[1][1:]]
formated_data = pd.DataFrame(dict_)
formated_data.head()

,Movie,User
0,1,37
1,1,73
2,1,156
3,1,160
4,1,248


In [14]:
pDF = sq.createDataFrame(formated_data)
spDF = pDF.rdd

In [15]:
testdata = spDF.map(lambda p: (p[0], p[1]))
predictions = model.predictAll(testdata).map(lambda r: (r[0], r[1], r[2]))


In [16]:
s  = predictions.toDF()

In [17]:
lol = s.toPandas()

In [18]:
lol.sort_values(['_1','_2'],ascending=[1,1],inplace=True)
predic_end = lol.reset_index(drop=True)

In [19]:
predic_end[predic_end._3 < 1 ] = 1
predic_end[predic_end._3 > 5 ] = 5
predic_end.head()

,_1,_2,_3
0,1,37,3.369211
1,1,73,3.070760
2,1,156,3.630625
3,1,160,3.304232
4,1,248,3.476489


In [ ]:
data_test.Prediction = predic_end._3
data_test.to_csv('submission.csv',index=False)